In [ ]:
import sys
import os
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from dotenv import load_dotenv
import importlib
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import pandas as pd
import requests
from src.visualisation import plot_team_scatter
from src import utils
from src.config import DATA_PATH

pd.set_option('display.max_columns', None)

Inspired by Christopher Davis and the Deep Dive Pod [Youtube link](https://www.youtube.com/watch?v=QvNcnS-2EOY&ab_channel=ChristopherDavis%2CUCIrvine)

In [ ]:
df = nfl.import_schedules([2023])
cols_of_interest = [
    'game_id',
    'week',
    'away_team', 
    'home_team',
    'total',
    'spread_line'
]
# weeks_of_interest = list(range(9, 18))
# df = schedule[schedule['week'].isin(weeks_of_interest)][cols_of_interest].copy()
df = df[cols_of_interest]
df = df[(df['week']<6)]
# df = df[(df['week']<6) & (df['week']>2)]

In [ ]:
df

In [ ]:
importlib.reload(utils)

In [ ]:
power_ratings, hfa = utils.derive_market_power_ratings(df)
print(hfa)
# power_ratings['adjusted'] = power_ratings['ovr'] - power_ratings['ovr'].mean()
power_ratings.round(1)


In [ ]:
plot_team_scatter(power_ratings, 'off', 'def', '2023 through week 5')

In [ ]:
griff = pd.read_clipboard()

In [ ]:
griff

In [ ]:
plot_team_scatter(griff, 'off', 'def', 'Griffin\'s Power Ratings through week 6')

# Odds API

In [ ]:

# Load environment variables from .env file
load_dotenv()

# Access the variables
API_KEY = os.getenv("API_KEY")
db_password = os.getenv("DB_PASSWORD")

SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'spreads,totals' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'american' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# sports_response = requests.get(
#     'https://api.the-odds-api.com/v4/sports', 
#     params={
#         'api_key': API_KEY
#     }
# )

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

In [ ]:
len(odds_json)

In [ ]:
len(odds_json[0])

In [ ]:
power_rating = {}
hfa = {}
for year in range(2018, 2023):
    pr, hf = derive_market_power_ratings(year)
    power_rating[year] = pr
    hfa[year] = hf
    _ = plot_team_scatter(pr, 'off', 'def', title=f'Market Derived Power Ratings - {year} Season - home field {hfa[year]}', ax_labels=('Offensive Power Rating', 'Defensive Power Rating'), mean_reference=False)